In [1]:
"""
[imports]
[def functions]
[programa]

1. Entrar al correo [inbox]
2. Guardar en un df todos los correos de un determinado periodo y cerrar sesionusername = #usuario de correo electrónico con forma xyz@correo.com password = '' #contraseña del usuario
3. filtrar en el df los correos de detectores
4. crear un df para fallas y otro para reposiciones
5. extraer la información necesaria
6. exportar a un csv/excel los dos df
"""


"\n[imports]\n[def functions]\n[programa]\n\n1. Entrar al correo [inbox]\n2. Guardar en un df todos los correos de un determinado periodo y cerrar sesionusername = #usuario de correo electrónico con forma xyz@correo.com password = '' #contraseña del usuario\n3. filtrar en el df los correos de detectores\n4. crear un df para fallas y otro para reposiciones\n5. extraer la información necesaria\n6. exportar a un csv/excel los dos df\n"

In [2]:
#[imports y definicion de variables]
import imaplib #importa módilo para ingresar a los correos
import re #módulo para hacer las busquedas de los textos que quiero extraer del body
import pandas as pd #módulo para trabajar con dataframes y exportarlo a hojas de cálculo
import email
import datetime
import time
from itertools import chain

direccion='MiDirectorio'

imap_ssl_host = 'imap.gmail.com'  # el host ssl del servidor de correo electrónico
imap_ssl_port = 993 #el puerto de acceso al host
username = '***' #usuario de correo electrónico con forma xyz@correo.com
password = '***' #contraseña del usuario
mes = "06/2018"

In [3]:
#[def functions]
def crear_data_frame(lista): # crear el primer df agregarla al df grande
    diccionario = {'from':[lista[0]],'subject':[lista[1]], 'date_loc':[lista[2]], 'body' : [lista[3]]} #convierte la lista en diccionario
    return pd.DataFrame(diccionario)

def aumentar_data_frame(lista, data_frame_final): # convertir la lista en df y agregarla al df grande
    diccionario = {'from':[lista[0]],'subject':[lista[1]], 'date_loc':[lista[2]], 'body' : [lista[3]]} #convierte la lista en diccionario
    data_frame_temporal= pd.DataFrame(diccionario)
    return pd.concat([data_frame_final,data_frame_temporal], ignore_index=True)

def otracosa(M,num):#retorna msg
    rv, data = M.fetch(num, '(RFC822)')
    if rv != 'OK':
        print("ERROR getting message", num)
        return

    return email.message_from_bytes(data[0][1])

def extraer_data_mail_to_list(M,num):#crear la lista de contenido por cada correo con: from, subject, fecha y body
    msg = otracosa(M,num)
    hdr = email.header.make_header(email.header.decode_header(msg['Subject']))
    subject = str(hdr)
    frm = email.header.make_header(email.header.decode_header(msg['From']))
    strfrom = str(frm)
    date_tuple = email.utils.parsedate_tz(msg['Date'])
    if date_tuple:
        local_date = datetime.datetime.fromtimestamp(
            email.utils.mktime_tz(date_tuple))
    
    status,datab = M.fetch(num,'(UID BODY[TEXT])')
    str_body = str(datab[0][1],'utf-8')
    return [strfrom,subject,local_date.strftime("%d/%m/%Y %H:%M:%S"), str_body]

def extraer_anio_y_mes(M,num):#retorna [mes,año] del mensaje
    msg = otracosa(M,num)
    date_tuple = email.utils.parsedate_tz(msg['Date'])
    if date_tuple:
        local_date = datetime.datetime.fromtimestamp(
            email.utils.mktime_tz(date_tuple))
    
    return local_date.strftime("%m/%Y")

def extraer_tipo_falla(str_data): #Encontrar el tipo de falla
    #str_data=str(data,'utf-8')
    pattern = re.compile(r'OSINERGMIN<br>([a-zA-Z0-9_.* ]*)<br>')
    matches = pattern.finditer(str_data)
    matches = pattern.finditer(str_data)
    lista_match = None
    for match in matches:
        if lista_match is None:
            lista_match = [match[1]]
        else:
            lista_match.append(match[1])
    
    if lista_match is None:
        lista_match=['---']
    
    return lista_match[0]

def extraer_ubicacion(str_data): #Encontrar el tipo de falla !!! 
    ###hay que ver la forma de que incluya espacios
    #str_data=str(data,'utf-8')
    pattern = re.compile(r'Ubicacion:([a-zA-Z0-9_.*\S ]*)<br>Fecha')
    matches = pattern.finditer(str_data)
    matches = pattern.finditer(str_data)
    lista_match = None
    for match in matches:
        if lista_match is None:
            lista_match = [match[1]]
        else:
            lista_match.append(match[1])
    
    if lista_match is None:
        lista_match=['---']
    
    return lista_match[0]

def extraer_fecha_ocurrencia(str_data): #Encontrar el tipo de falla !!! 
    ###hay que ver la forma de que incluya espacios
    #str_data=str(data,'utf-8')
    pattern = re.compile(r'Fecha de ocurrencia:([a-zA-Z0-9_.*\W ]*)<br>Hora')
    matches = pattern.finditer(str_data)
    matches = pattern.finditer(str_data)
    lista_match = None
    for match in matches:
        if lista_match is None:
            lista_match = [match[1]]
        else:
            lista_match.append(match[1])
    
    if lista_match is None:
        lista_match=['---']
    
    return lista_match[0]


def extraer_hora_ocurrencia(str_data): #Encontrar el tipo de falla !!! 
    ###hay que ver la forma de que incluya espacios
    #str_data=str(data,'utf-8')
    pattern = re.compile(r'<br>Hora de ocurrencia:([a-zA-Z0-9_.*\W ]*)h<br>')
    matches = pattern.finditer(str_data)
    matches = pattern.finditer(str_data)
    lista_match = None
    for match in matches:
        if lista_match is None:
            lista_match = [match[1]]
        else:
            lista_match.append(match[1])
    
    if lista_match is None:
        lista_match=['---']
    
    return lista_match[0]

def extraer_numero_medidor(str_data): #Encontrar el tipo de falla !!! 
    ###hay que ver la forma de que incluya espacios
    #str_data=str(data,'utf-8')
    pattern = re.compile(r'<br>Medidor N:([a-zA-Z0-9_.*\W ]*)<br>Ubicacion:')
    matches = pattern.finditer(str_data)
    matches = pattern.finditer(str_data)
    lista_match = None
    for match in matches:
        if lista_match is None:
            lista_match = [match[1]]
        else:
            lista_match.append(match[1])
    
    if lista_match is None:
        lista_match=['---']
    
    return lista_match[0]


def extraer_duracion(str_data): #Encontrar el tipo de falla !!! 
    ###hay que ver la forma de que incluya espacios
    #str_data=str(data,'utf-8')
    pattern = re.compile(r'<br>Duracion:([a-zA-Z0-9_.*\W ]*)<br></p>')
    matches = pattern.finditer(str_data)
    matches = pattern.finditer(str_data)
    lista_match = None
    for match in matches:
        if lista_match is None:
            lista_match = [match[1]]
        else:
            lista_match.append(match[1])
    
    if lista_match is None:
        lista_match=['---']
    
    return lista_match[0]




In [4]:
#[programa]
#1. Entrar al correo
#2. Guardar en un df todos los correos de un determinado periodo y cerrar sesion
M=imaplib.IMAP4_SSL(imap_ssl_host,imap_ssl_port)
M.login(username,password)
status,count = M.select('Inbox')
print(status,count,'entraste a la bandeja de entrada')

rv, data = M.search(None, '(FROM "soporte@eccion.pe")')
print(rv, 'la busqueda salión bien')
data_frame_final = None
for num in data[0].split():
    b = extraer_anio_y_mes(M,num)
    if b==mes :
        print(mes, ".l.")
        lista = extraer_data_mail_to_list(M,num)
        try: data_frame_final
        except NameError: data_frame_final = None

        if data_frame_final is None:
            data_frame_final = crear_data_frame(lista)
        else:
            data_frame_final = aumentar_data_frame(lista, data_frame_final)

            
M.logout()

OK [b'1828'] entraste a la puta bandeja de entrada
OK la puta busqueda salión bien
06/2018 .l.
06/2018 .l.
06/2018 .l.
06/2018 .l.
06/2018 .l.
06/2018 .l.
06/2018 .l.
06/2018 .l.
06/2018 .l.
06/2018 .l.
06/2018 .l.
06/2018 .l.
06/2018 .l.
06/2018 .l.
06/2018 .l.
06/2018 .l.
06/2018 .l.
06/2018 .l.
06/2018 .l.
06/2018 .l.
06/2018 .l.
06/2018 .l.
06/2018 .l.
06/2018 .l.
06/2018 .l.
06/2018 .l.
06/2018 .l.
06/2018 .l.
06/2018 .l.
06/2018 .l.
06/2018 .l.
06/2018 .l.
06/2018 .l.
06/2018 .l.
06/2018 .l.
06/2018 .l.
06/2018 .l.
06/2018 .l.
06/2018 .l.
06/2018 .l.
06/2018 .l.
06/2018 .l.
06/2018 .l.
06/2018 .l.
06/2018 .l.
06/2018 .l.
06/2018 .l.
06/2018 .l.
06/2018 .l.
06/2018 .l.
06/2018 .l.
06/2018 .l.
06/2018 .l.
06/2018 .l.


('BYE', [b'LOGOUT Requested'])

In [11]:
#[programa]
#3. agregar los datos del body a columnas del df

df=data_frame_final
df['tipo_falla']=df['body'].apply(lambda x: extraer_tipo_falla(x))
df['medidor']=df['body'].apply(lambda x: extraer_numero_medidor(x))
df['ubicacion']=df['body'].apply(lambda x: extraer_ubicacion(x))
df['fecha_ocurrencia']=df['body'].apply(lambda x: extraer_fecha_ocurrencia(x))
df['hora_ocurrencia']=df['body'].apply(lambda x: extraer_hora_ocurrencia(x))
df['duracion']=df['body'].apply(lambda x: extraer_duracion(x))
df_excel = df[df.columns[1:]]
df_excel

,date_loc,from,subject,tipo_falla,medidor,ubicacion,fecha_ocurrencia,hora_ocurrencia,duracion
0,01/06/2018 06:41:14,soporte@eccion.pe,(OSINERGMIN) AVISO FALLA GENERAL - Electro Nor...,FALLA GENERAL,73069778,ENO_ARENAL_A1029_E160144_La Huaca - Miraflores,01-06-2018,06:40,---
1,01/06/2018 06:41:33,soporte@eccion.pe,(OSINERGMIN) AVISO FALLA GENERAL - Electro Nor...,FALLA GENERAL,73069776,ENO_ARENAL_A1029_E160136_La Huaca - Pucusula,01-06-2018,06:40,---
2,01/06/2018 08:19:08,soporte@eccion.pe,(OSINERGMIN) AVISO REPOSICION GENERAL - Electr...,REPOSICION GENERAL,73069778,ENO_ARENAL_A1029_E160144_La Huaca - Miraflores,01-06-2018,08:19,"1 horas, 37 minutos, 50 segundos"
3,01/06/2018 08:52:01,soporte@eccion.pe,(OSINERGMIN) AVISO FALLA GENERAL - Electro Ori...,FALLA GENERAL,73069740,EOR_BAGUA-JAEN RURAL_A2404_SED SE51_Ca. Sucre,01-06-2018,08:51,---
4,01/06/2018 08:52:19,soporte@eccion.pe,(OSINERGMIN) AVISO FALLA GENERAL - Electro Ori...,FALLA GENERAL,73069788,EOR_BAGUA-JAEN RURAL_A2404_SED SE48_Ca. Lamba...,01-06-2018,08:51,---
5,01/06/2018 08:53:32,soporte@eccion.pe,(OSINERGMIN) AVISO REPOSICION GENERAL - Electr...,REPOSICION GENERAL,73069740,EOR_BAGUA-JAEN RURAL_A2404_SED SE51_Ca. Sucre,01-06-2018,08:53,"0 horas, 1 minutos, 32 segundos"
6,01/06/2018 08:55:01,soporte@eccion.pe,(OSINERGMIN) AVISO REPOSICION GENERAL - Electr...,REPOSICION GENERAL,73069788,EOR_BAGUA-JAEN RURAL_A2404_SED SE48_Ca. Lamba...,01-06-2018,08:54,"0 horas, 2 minutos, 49 segundos"
7,01/06/2018 12:27:48,soporte@eccion.pe,(OSINERGMIN) AVISO REPOSICION GENERAL - Electr...,REPOSICION GENERAL,73069776,ENO_ARENAL_A1029_E160136_La Huaca - Pucusula,01-06-2018,12:27,"5 horas, 46 minutos, 33 segundos"
8,01/06/2018 12:28:13,soporte@eccion.pe,(OSINERGMIN) AVISO REPOSICION GENERAL - Electr...,REPOSICION GENERAL,73069776,ENO_ARENAL_A1029_E160136_La Huaca - Pucusula,01-06-2018,12:28,"5 horas, 47 minutos, 1 segundos"
9,01/06/2018 12:55:10,soporte@eccion.pe,(OSINERGMIN) AVISO REPOSICION GENERAL - Electr...,REPOSICION GENERAL,73069755,ELN_CHONGOYAPE_A2056_SED 3620_Huambara,01-06-2018,12:55,"16 horas, 46 minutos, 32 segundos"


In [12]:
#[programa]
#4. exportar en excel
guardar=str(direccion)+'/Charles_es_Pendejo/'+str(mes).replace('/','.')+'.xlsx'
df_excel.to_excel(guardar)
print('guardaro en:',guardar)

guardaro en: C:/Users/cueva/Downloads/Charles_es_Pendejo/06.2018.xlsx
